# 🗣️ Word Embeddings
## *Teaching computers what words mean*

---

How does ChatGPT know that **"dog"** and **"puppy"** are similar?

Or that **"Paris"** is to **"France"** as **"Tokyo"** is to **"Japan"**?

The answer: **word embeddings** — representing every word as a list of numbers (a *vector*) so that words with similar meanings end up close together in space.

```
king   → [0.32, -0.51, 0.78, 0.14, ...50 numbers...]
queen  → [0.30, -0.48, 0.75, 0.19, ...]
dog    → [-0.62, 0.33, -0.10, 0.88, ...]
```

This is the **foundation of all modern AI language models** — ChatGPT, Gemini, Claude, all of them.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import seaborn as sns
import ipywidgets as widgets
from sklearn.decomposition import PCA

sns.set_theme(style="white", font_scale=1.15)

# Load the pre-sampled GloVe-style vectors
df = pd.read_csv("../data/glove_sample.csv")
words      = df["word"].tolist()
categories = df["category"].tolist()
vectors    = df.drop(columns=["word", "category"]).values  # shape: (80, 50)

print(f"Loaded {len(words)} words across {df['category'].nunique()} categories.")
print("Categories:", sorted(df["category"].unique()))

## What is a Vector?

Think of it as **coordinates** for meaning.

In 2D, we can place words on a map. In 50D, we can capture much richer relationships.

**Cosine similarity** measures how *similar* two word-vectors point:
- Score **1.0** = identical direction (same meaning)
- Score **0.0** = perpendicular (unrelated)
- Score **-1.0** = opposite direction (antonyms)

In [ ]:
def cosine_similarity(v1: np.ndarray, v2: np.ndarray) -> float:
    return float(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))


def get_vector(word: str) -> np.ndarray:
    idx = words.index(word)
    return vectors[idx]


def nearest_neighbors(word: str, n: int = 5) -> list[tuple[str, float]]:
    v   = get_vector(word)
    sims = [(w, cosine_similarity(v, vectors[i]))
            for i, w in enumerate(words) if w != word]
    return sorted(sims, key=lambda x: x[1], reverse=True)[:n]

## Interactive Demo 1 — Nearest Neighbors

Pick a word and see which other words are most similar to it.

In [ ]:
@widgets.interact(
    word=widgets.Dropdown(options=sorted(words), value="king", description="Word:",
                          layout=widgets.Layout(width="250px")),
    n=widgets.IntSlider(value=5, min=3, max=10, step=1, description="Show top:",
                        style={"description_width": "initial"},
                        layout=widgets.Layout(width="300px")),
)
def show_neighbors(word, n):
    neighbors = nearest_neighbors(word, n)
    neighbor_words  = [w for w, _ in neighbors]
    neighbor_scores = [s for _, s in neighbors]

    fig, ax = plt.subplots(figsize=(8, max(3, n * 0.55)))

    colors = plt.cm.RdYlGn(np.array(neighbor_scores))
    bars = ax.barh(range(n), neighbor_scores, color=colors, edgecolor="white", linewidth=1)
    ax.set_yticks(range(n))
    ax.set_yticklabels(neighbor_words, fontsize=12)
    ax.invert_yaxis()
    ax.set_xlabel("Cosine Similarity")
    ax.set_title(f'Words most similar to "{word}"', fontsize=14, weight="bold")
    ax.set_xlim(0, 1.05)
    ax.bar_label(bars, fmt="%.3f", padding=4, fontsize=11, fontweight="bold")

    cat = categories[words.index(word)]
    ax.set_xlabel(f'Cosine Similarity   (category: {cat})', fontsize=11)

    sns.despine(left=True, bottom=True)
    plt.tight_layout()
    plt.show()

## Interactive Demo 2 — Word Map (PCA)

50 dimensions is impossible to visualize. We use **PCA (Principal Component Analysis)** to compress
50 dimensions down to 2 while preserving as much structure as possible.

Words with similar meanings should cluster together on the map.

In [ ]:
cat_palette = {
    "royalty":   "#E8575A",
    "animals":   "#5B8FB9",
    "food":      "#F4A261",
    "tech":      "#6BCB77",
    "sports":    "#9B5DE5",
    "emotions":  "#FF6B9D",
    "countries": "#00B4D8",
    "science":   "#E9C46A",
}

pca     = PCA(n_components=2, random_state=42)
coords  = pca.fit_transform(vectors)
var_exp = pca.explained_variance_ratio_

@widgets.interact(
    cats=widgets.SelectMultiple(
        options=sorted(cat_palette.keys()),
        value=tuple(sorted(cat_palette.keys())),
        description="Categories:",
        layout=widgets.Layout(height="160px", width="220px"),
    ),
    show_labels=widgets.Checkbox(value=True, description="Show word labels"),
)
def plot_word_map(cats, show_labels):
    fig, ax = plt.subplots(figsize=(11, 8))

    for cat in cats:
        mask = np.array([c == cat for c in categories])
        ax.scatter(coords[mask, 0], coords[mask, 1],
                   c=cat_palette[cat], s=100, label=cat.capitalize(),
                   edgecolors="white", linewidths=0.8, zorder=3)
        if show_labels:
            for i, (x, y) in enumerate(coords[mask]):
                w = np.array(words)[mask][i]
                ax.text(x + 0.015, y, w, fontsize=8.5, color=cat_palette[cat],
                        path_effects=[pe.withStroke(linewidth=2, foreground="white")])

    ax.set_xlabel(f"PC 1 ({var_exp[0]*100:.1f}% variance)", fontsize=11)
    ax.set_ylabel(f"PC 2 ({var_exp[1]*100:.1f}% variance)", fontsize=11)
    ax.set_title("Word Embedding Map — 50 Dimensions → 2D via PCA",
                 fontsize=14, weight="bold")
    ax.legend(title="Category", bbox_to_anchor=(1.01, 1), loc="upper left",
              fontsize=10, title_fontsize=11)
    ax.grid(True, alpha=0.3)
    sns.despine()
    plt.tight_layout()
    plt.show()

## The Word Arithmetic

One of the most surprising properties of word vectors:

$$\text{king} - \text{man} + \text{woman} \approx \text{queen}$$

You can do **arithmetic with meanings**!

In [ ]:
royalty_words  = [w for w, c in zip(words, categories) if c == "royalty"]

@widgets.interact(
    a=widgets.Dropdown(options=royalty_words,  value="queen",   description="Word A:",
                       layout=widgets.Layout(width="200px")),
    b=widgets.Dropdown(options=royalty_words,  value="king",  description="- Word B:",
                       layout=widgets.Layout(width="200px")),
    c=widgets.Dropdown(options=royalty_words,  value="prince", description="+ Word C:",
                       layout=widgets.Layout(width="200px")),
)
def word_arithmetic(a, b, c):
    result_vec = get_vector(a) - get_vector(b) + get_vector(c)
    result_vec /= np.linalg.norm(result_vec)

    sims = [(w, cosine_similarity(result_vec, vectors[i]))
            for i, w in enumerate(words) if w not in [a, b, c]]
    top5 = sorted(sims, key=lambda x: x[1], reverse=True)[:5]

    print(f'  "{a}" - "{b}" + "{c}" = ?')
    print()
    for rank, (w, s) in enumerate(top5, 1):
        bar = "█" * int(s * 30)
        print(f"  {rank}. {w:<15} similarity: {s:.3f}  {bar}")

## Key Takeaways

1. **Words are numbers.** Every word in a language model is represented as a list of numbers (over 4000 for large models).
2. **Similar meanings → similar vectors.** The model learns this by reading billions of sentences.
3. **PCA** lets us squish high-dimensional data into 2D so humans can see the structure.
4. **Word arithmetic** (king − man + woman ≈ queen) shows that vector spaces capture real-world relationships.

This is the core idea behind **every large language model** — GPT-5, Gemini, Claude. They all start with word embeddings.
